# Amazon Bedrock으로 채팅 완료 요청을 스트리밍하는 방법

기본적으로 완료를 요청하면 전체 완료가 생성된 후 단일 응답으로 다시 전송됩니다.

긴 완료를 생성하는 경우 응답을 기다리는 데 몇 초가 걸릴 수 있습니다.

응답을 더 빨리 받으려면 완료가 생성되는 동안 '스트리밍'할 수 있습니다. 이렇게 하면 전체 완료가 끝나기 전에 완료의 시작 부분을 인쇄하거나 처리할 수 있습니다.

완료를 스트리밍하려면 채팅 완료 또는 완료 엔드포인트를 호출할 때 `stream=True`를 설정합니다. 그러면 [데이터 전용 서버 전송 이벤트](https://developer.mozilla.org/en-US/docs/Web/API/Server-sent_events/Using_server-sent_events#event_stream_format)로 응답을 다시 스트리밍하는 개체가 반환됩니다. `message` 필드 대신 `delta` 필드에서 청크를 추출합니다.

## 단점

프로덕션 애플리케이션에서 `stream=True`를 사용하면 부분 완료를 평가하기가 더 어려울 수 있으므로 완료 내용의 조정이 더 어려워집니다.

## 설정

Amazon Bedrock에서 `easyllm`을 사용하려면 먼저 모델에 대한 권한 및 액세스를 설정해야 합니다. 아래 지침에 따라 이 작업을 수행할 수 있습니다.
* https://docs.aws.amazon.com/IAM/latest/UserGuide/getting-set-up.html
* https://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html
* https://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html

## 예제 코드

아래에서 이 노트북은 다음을 보여줍니다.
1. 일반적인 채팅 완료 응답의 모양
2. 스트리밍 채팅 완료 응답의 모양
3. 채팅 완료를 스트리밍하여 절약되는 시간

In [ ]:
# 필요한 경우 EasyLLM Python 라이브러리의 최신 버전으로 설치 및/또는 업그레이드합니다.
%pip install --upgrade easyllm[bedrock] 

In [1]:
# 가져오기
import easyllm  # API 호출용

### 1. 일반적인 채팅 완료 응답의 모양

일반적인 ChatCompletions API 호출을 사용하면 응답이 먼저 계산된 다음 한 번에 모두 반환됩니다.

In [1]:
import os 
# 프롬프트 빌더용 환경 변수 설정
os.environ["BEDROCK_PROMPT"] = "anthropic" # vicuna, wizardlm, stablebeluga, open_assistant
os.environ["AWS_REGION"] = "us-east-1"  # 해당 지역으로 변경
# os.environ["AWS_ACCESS_KEY_ID"] = "XXX" # boto3 세션을 사용하지 않는 경우 필요
# os.environ["AWS_SECRET_ACCESS_KEY"] = "XXX" # boto3 세션을 사용하지 않는 경우 필요

from easyllm.clients import bedrock

response = bedrock.ChatCompletion.create(
    model='anthropic.claude-v2',
    messages=[
        {'role': 'user', 'content': '100까지 세되 각 숫자 사이에 쉼표를 넣고 줄 바꿈은 하지 마세요. 예: 1, 2, 3, ...'}
    ],
    stream=True
)

for chunk in response:
    print(chunk)


10/26/2023 17:34:57 - INFO - easyllm.utils.logging - boto3 Bedrock 클라이언트가 성공적으로 생성되었습니다!
{'id': 'hf-Je8BGADPWN', 'object': 'chat.completion.chunk', 'created': 1698334497, 'model': 'anthropic.claude-v2', 'choices': [{'index': 0, 'delta': {'role': 'assistant'}}]}
{'id': 'hf-Je8BGADPWN', 'object': 'chat.completion.chunk', 'created': 1698334498, 'model': 'anthropic.claude-v2', 'choices': [{'index': 0, 'delta': {'content': ' 여기'}}]}
{'id': 'hf-Je8BGADPWN', 'object': 'chat.completion.chunk', 'created': 1698334498, 'model': 'anthropic.claude-v2', 'choices': [{'index': 0, 'delta': {'content': ' 각 숫자 사이에 쉼표를 넣고 줄 바꿈 없이 100까지 셉니다:\n\n1, 2, 3,'}}]}
{'id': 'hf-Je8BGADPWN', 'object': 'chat.completion.chunk', 'created': 1698334499, 'model': 'anthropic.claude-v2', 'choices': [{'index': 0, 'delta': {'content': ' 4, 5, 6, 7, 8, 9, 10, 11'}}]}
{'id': 'hf-Je8BGADPWN', 'object': 'chat.completion.chunk', 'created': 1698334499, 'model': 'anthropic.claude-v2', 'choices': [{'index': 0, 'delta': {'content': ',

위에서 볼 수 있듯이 스트리밍 응답에는 `message` 필드 대신 `delta` 필드가 있습니다. `delta`는 다음과 같은 항목을 포함할 수 있습니다.
- 역할 토큰 (예: `{"role": "assistant"}`)
- 콘텐츠 토큰 (예: `{"content": "\n\n"}`)
- 스트림이 끝나면 아무것도 없음 (예: `{}`)

### 3. 채팅 완료를 스트리밍하여 절약되는 시간

이제 `meta-llama/Llama-2-70b-chat-hf`에게 다시 100까지 세도록 요청하고 얼마나 걸리는지 확인해 보겠습니다.

In [7]:
import os 
# 프롬프트 빌더용 환경 변수 설정
os.environ["BEDROCK_PROMPT"] = "anthropic" # vicuna, wizardlm, stablebeluga, open_assistant
os.environ["AWS_REGION"] = "us-east-1"  # 해당 지역으로 변경
os.environ["AWS_PROFILE"] = "hf-sm"  # 해당 지역으로 변경
# os.environ["AWS_ACCESS_KEY_ID"] = "XXX" # boto3 세션을 사용하지 않는 경우 필요
# os.environ["AWS_SECRET_ACCESS_KEY"] = "XXX" # boto3 세션을 사용하지 않는 경우 필요
from easyllm.clients import bedrock

# 100까지 세는 ChatCompletion 요청 보내기
response = bedrock.ChatCompletion.create(
    model='anthropic.claude-v2',
    messages=[
        {'role': 'user', 'content': '100까지 세되 각 숫자 사이에 쉼표를 넣고 줄 바꿈은 하지 마세요. 예: 1, 2, 3, ...'}
    ],
    stream=True
)

# 청크 스트림을 수집하기 위한 변수 만들기
collected_chunks = []
collected_messages = []
# 이벤트 스트림 반복
for chunk in response:
    collected_chunks.append(chunk)  # 이벤트 응답 저장
    chunk_message = chunk['choices'][0]['delta']  # 메시지 추출
    print(chunk_message.get('content', ''), end='')  # 메시지 인쇄
    collected_messages.append(chunk_message)  # 메시지 저장
    

# 시간 지연 및 수신된 텍스트 인쇄
full_reply_content = ''.join([m.get('content', '') for m in collected_messages])
print(f"전체 대화 수신: {full_reply_content}")


 쉼표를 넣고 줄 바꿈 없이 100까지 셉니다:

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100전체 대화 수신:  쉼표를 넣고 줄 바꿈 없이 100까지 셉니다:

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100
